<a href="https://colab.research.google.com/github/aleehpandita/A01794334/blob/main/Equipo24_semana09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Procesamiento de Lenguaje Natural**

### Tecnológico de Monterrey

### Prof Luis Eduardo Falcón Morales

## Adtividad Semana 9

### **Traducciones y Clasificación de Textos**

#### **Nombres y matrículas de los integrantes del equipo:**

<table>
  <tr>
    <th>
      Nombre
    </th>
    <th>
      Matrícula
    </th>
  </tr>
  <tr>
    <td>
      Carlos Mario Larrahondo Valencia
    </td>
    <td>
      A01794031
    </td>
  </tr>
  <tr>
    <td>
      Alejandra Guadalupe Larranaga Altamirano
    </td>
    <td>
      A01794334
    </td>
  </tr>
  <tr>
    <td>
      Oscar Israel Lerma Franco
    </td>
    <td>
      A01380817
    </td>
  </tr>
  <tr>
    <td>
      Diego Alí López Álvarez
    </td>
    <td>
      A00817381
    </td>
  </tr>
  <tr>
    <td>
      Erik López Hernández
    </td>
    <td>
      A00458875
    </td>
  </tr>
</table>

In [ ]:
!pip install datasets transformers huggingface_hub


In [ ]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1.  Descarga el archivo amazonbaby5000.csv que se encuentra en Canvas. Este archivo está formado de 5 mil comentarios en inglés sobre productos para bebé adquiridos a través de la plataforma de Amazon.

In [ ]:
#("/content/drive/MyDrive/amazonbaby5000.csv")

import pandas as pd
from sklearn.model_selection import train_test_split

amazon_baby=pd.read_csv("/content/drive/MyDrive/amazonbaby5000.csv")
df = pd.DataFrame(amazon_baby)
df.head()



,review,rating
0,I think it is more Expensive than drugstore th...,0
1,"When I saw this on Amazon, I put it into my wi...",1
2,We really like these valances. They have such...,1
3,No light emits from the night light. They pain...,0
4,I was really hoping for this to be a conventie...,0


2. Realiza una partición de los datos en el porcentaje que consideres adecuado, en entrenamiento y prueba

In [ ]:
from datasets import Dataset

df_processed = df[['review', 'rating']]

# Convert pandas DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df_processed)

# Split the dataset into train and test sets
small_train_dataset = dataset.shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = dataset.shuffle(seed=42).select([i for i in list(range(300))])


# Inspect the Dataset
print(small_test_dataset)

Dataset({
    features: ['review', 'rating'],
    num_rows: 300
})


3. Utiliza un modelo Transformer de HuggingFace para análisis de sentimiento en inglés y lleva a cabo la predicción de los comentarios en los siguientes casos:

  a. Utiliza uno de los modelos Transformer pre-entrenados que consideres adecuado de DistilBERT de Huggingface para análisis de sentimiento y lleva a cabo el entrenamiento y evaluación del desempeño con exatitud (accuracy) y matriz de confusión.
  NOTA: Recuerda obtener el porcentaje de las clases positivas y negativas para tener una idea clara de si el modelo encontrado queda subentrenado.


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Define a preprocessing function to tokenize the text
def preprocess_function(examples):
    return tokenizer(examples['review'], padding='max_length', truncation=True)

# Tokenize the dataset
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

# Ensure labels are integers and included
# tokenized_train = tokenized_train.map(lambda x: {'labels': x['review']})
# tokenized_test = tokenized_test.map(lambda x: {'labels': x['review']})

# Rename 'label' column to 'labels' as expected by the model
# tokenized_train = tokenized_train.rename_column("rating", "labels")
# tokenized_test = tokenized_test.rename_column("rating", "labels")

# Ensure labels are included
tokenized_train = tokenized_train.map(lambda x: {'labels': x['rating']})
tokenized_test = tokenized_test.map(lambda x: {'labels': x['rating']})



# Set the format of the dataset to return tensors
tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# let's define the metrics you will be using to evaluate how good is your fine-tuned model (accuracy and f1 score):
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}


In [18]:
#login to huggingface account
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install accelerate -U

In [ ]:
!pip install transformers[torch] datasets accelerate -U


In [ ]:
#Define the training arguments and define a Trainer with all the objects you constructed up to this point
from transformers import TrainingArguments, Trainer


In [20]:
repo_name = "amazon-baby-distilbert"
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [22]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

trainer.train()
results = trainer.evaluate()
print(results)

# Imprimimos los resultados
print(f"Accuracy: {results['eval_accuracy']}")
print(f"F1 Score: {results['eval_f1']}")

# Obtenemos las predicciones para el conjunto de prueba
predictions = trainer.predict(tokenized_test)
preds = np.argmax(predictions.predictions, axis=-1)

# Calculamos y mostramos la matriz de confusión y el reporte de clasificación
cm = confusion_matrix(tokenized_test['labels'].numpy(), preds)
report = classification_report(tokenized_test['labels'].numpy(), preds)

print(f"Confusion Matrix: \n{cm}")
print(f"Classification Report: \n{report}")


test_labels = tokenized_test['labels'].numpy()
positive_percentage = sum(test_labels) / len(test_labels) * 100
negative_percentage = 100 - positive_percentage

print(f"Percentage of positive class: {positive_percentage}%")
print(f"Percentage of negative class: {negative_percentage}%")

Step,Training Loss


<ipython-input-10-aca103021d4f>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  load_accuracy = load_metric("accuracy")


{'eval_loss': 0.18674372136592865, 'eval_accuracy': 0.95, 'eval_f1': 0.9668874172185431, 'eval_runtime': 5.3103, 'eval_samples_per_second': 56.494, 'eval_steps_per_second': 3.578, 'epoch': 2.0}
Accuracy: 0.95
F1 Score: 0.9668874172185431
Confusion Matrix: 
[[ 66   5]
 [ 10 219]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.87      0.93      0.90        71
           1       0.98      0.96      0.97       229

    accuracy                           0.95       300
   macro avg       0.92      0.94      0.93       300
weighted avg       0.95      0.95      0.95       300

Percentage of positive class: 76.33333333333333%
Percentage of negative class: 23.66666666666667%
